Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16    # Size of a batch, nothing to do with size of convolutional layers
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, 
                                      shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
                                [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
                                [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
                                [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
                                [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        '''
        tf.nn.conv2d(input, filter, strides, padding,use_cudnn_on_gpu=None, data_format=None, name=None)
        
        input size: 16 x 28 x 28 x 1
        filter size: 5 x 5 x 1 x 16
        '''
            
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME') # strides = 2
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1000

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps+1):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 100 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.780216
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 100: 1.006960
Minibatch accuracy: 68.8%
Validation accuracy: 70.3%
Minibatch loss at step 200: 0.916399
Minibatch accuracy: 68.8%
Validation accuracy: 77.6%
Minibatch loss at step 300: 0.346500
Minibatch accuracy: 87.5%
Validation accuracy: 79.1%
Minibatch loss at step 400: 0.225123
Minibatch accuracy: 100.0%
Validation accuracy: 79.4%
Minibatch loss at step 500: 0.782777
Minibatch accuracy: 87.5%
Validation accuracy: 81.1%
Minibatch loss at step 600: 0.310264
Minibatch accuracy: 87.5%
Validation accuracy: 81.9%
Minibatch loss at step 700: 0.837835
Minibatch accuracy: 75.0%
Validation accuracy: 81.8%
Minibatch loss at step 800: 0.711324
Minibatch accuracy: 75.0%
Validation accuracy: 82.5%
Minibatch loss at step 900: 0.589534
Minibatch accuracy: 93.8%
Validation accuracy: 82.8%
Minibatch loss at step 1000: 0.435456
Minibatch accuracy: 87.5%
Validation accuracy: 82.4

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [7]:
batch_size = 16    # Size of a batch, nothing to do with size of convolutional layers
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, 
                                      shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
                                [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
                                [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    # "// 4" here because of the two 2-stride
    layer3_weights = tf.Variable(tf.truncated_normal(
                                [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
                                [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):    
        '''
        The sizes here are very messy, and the good way to understand the whole CNN thing
        might just be to understand the sizes.
        The kernel and the stride sizes (and the padding type??) determine the layer3 size.
        For layer3_weights, "image_size // 4 * image_size // 4 * depth" is because we have
        a kernel size of 2 and a stride size of 2, so eventually the size has to be floor
        divided by 4.
        '''
        # Layer1: convolution, rectifier, and polling
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding = 'SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        pool = tf.nn.max_pool(hidden, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
        
        # Layer2: convolution, rectifier, and polling
        conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding = 'SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        pool = tf.nn.max_pool(hidden, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
        
        # Layer3: fully connected
        shape = pool.get_shape().as_list()
        reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [8]:
num_steps = 1000

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps+1):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 100 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.536790
Minibatch accuracy: 12.5%
Validation accuracy: 11.2%
Minibatch loss at step 100: 1.176435
Minibatch accuracy: 56.2%
Validation accuracy: 57.5%
Minibatch loss at step 200: 1.118431
Minibatch accuracy: 62.5%
Validation accuracy: 77.2%
Minibatch loss at step 300: 0.347405
Minibatch accuracy: 87.5%
Validation accuracy: 80.5%
Minibatch loss at step 400: 0.256048
Minibatch accuracy: 100.0%
Validation accuracy: 80.5%
Minibatch loss at step 500: 0.707609
Minibatch accuracy: 87.5%
Validation accuracy: 82.1%
Minibatch loss at step 600: 0.376036
Minibatch accuracy: 87.5%
Validation accuracy: 82.8%
Minibatch loss at step 700: 0.907936
Minibatch accuracy: 68.8%
Validation accuracy: 82.2%
Minibatch loss at step 800: 0.551776
Minibatch accuracy: 87.5%
Validation accuracy: 83.8%
Minibatch loss at step 900: 0.557421
Minibatch accuracy: 81.2%
Validation accuracy: 84.3%
Minibatch loss at step 1000: 0.460221
Minibatch accuracy: 81.2%
Validation accuracy: 84.1

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [22]:
batch_size = 16    # Size of a batch, nothing to do with size of convolutional layers
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, 
                                      shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta = tf.placeholder(tf.float32)
    
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
                                [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
                                [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
                                [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
                                [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    global_step = tf.Variable(0)
    # Model.
    def model(data, train = False):    
        # Layer1: convolution, rectifier, and polling
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding = 'SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        pool = tf.nn.max_pool(hidden, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
        
        # Layer2: convolution, rectifier, and polling
        conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding = 'SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        pool = tf.nn.max_pool(hidden, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
        
        # Layer3: fully connected
        shape = pool.get_shape().as_list()
        reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        
        # Dropout
        if (train):
            hidden = tf.nn.dropout(hidden, 0.5)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
        
    # Training computation.
    logits = model(tf_train_dataset, True)
    
    # Regulation
    regul = beta * (tf.nn.l2_loss(layer1_weights) +
                    tf.nn.l2_loss(layer2_weights) +
                    tf.nn.l2_loss(layer3_weights) +
                    tf.nn.l2_loss(layer4_weights))
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) + regul)
    
    # Optimizer.
    learning_rate = tf.train.exponential_decay(0.05, global_step, 1000, 0.85, staircase = True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [23]:
num_steps = 3000

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps+1):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta : 1e-2}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 200 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 6.808399
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 200: 3.519289
Minibatch accuracy: 43.8%
Validation accuracy: 42.6%
Minibatch loss at step 400: 2.532730
Minibatch accuracy: 50.0%
Validation accuracy: 76.2%
Minibatch loss at step 600: 1.913727
Minibatch accuracy: 81.2%
Validation accuracy: 80.6%
Minibatch loss at step 800: 1.954945
Minibatch accuracy: 68.8%
Validation accuracy: 81.4%
Minibatch loss at step 1000: 1.544039
Minibatch accuracy: 87.5%
Validation accuracy: 82.0%
Minibatch loss at step 1200: 2.168025
Minibatch accuracy: 56.2%
Validation accuracy: 82.9%
Minibatch loss at step 1400: 1.247205
Minibatch accuracy: 87.5%
Validation accuracy: 83.1%
Minibatch loss at step 1600: 1.943437
Minibatch accuracy: 75.0%
Validation accuracy: 83.7%
Minibatch loss at step 1800: 1.068961
Minibatch accuracy: 75.0%
Validation accuracy: 84.1%
Minibatch loss at step 2000: 0.745570
Minibatch accuracy: 93.8%
Validation accuracy: 8